In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import io
%matplotlib inline

def scatterplot(data, x, y, **kwargs):
    # Cirrus has an old version of Seaborn - work around lack of sns.scatterplot
    if sns.__version__ < '0.9':
        grid = sns.FacetGrid(data, **kwargs)
        grid.map(plt.scatter, x, y).add_legend()
        return grid
    return sns.scatterplot(data=data, x=x, y=y, **kwargs)


def munge_csv_datasets(*dsets):
    ans = []
    for raw in dsets:
        new = pd.read_csv(io.StringIO(raw), sep=', *',engine='python')
        new['nrows'] = 2**new['N']
        new['ncols'] = 2**new['M']
        new.drop(columns=['N', 'M'])
        ans.append(new)
    return pd.concat(ans)

In [ ]:
# Example data - replace with your measurements!
# CPU results
raw_cpu = '''exec_space, layout, cores, N, M, bandwidth
OpenMP, default, 10, 8, 14, 45.981
OpenMP, default, 10, 10, 12, 45.9073
OpenMP, default, 10, 12, 10, 45.8484
OpenMP, default, 10, 14, 8, 45.8484
OpenMP, default, 10, 16, 6, 43.5089
'''
# GPU results. NB: 84 SMs per GV100
raw_gpu = '''exec_space,layout, cores, N, M, bandwidth
CudaUVM, default, 84, 8, 14, 10.9454
CudaUVM, default, 84, 10, 12, 37.5838
CudaUVM, default, 84, 12, 10, 101.408
CudaUVM, default, 84, 14, 8, 179.82
CudaUVM, default, 84, 16, 6, 214.254
'''

data = munge_csv_datasets(raw_cpu, raw_gpu)

In [ ]:
fig = scatterplot(data, 'nrows', 'bandwidth', col='exec_space', hue='layout')
fig.set(xscale='log')